# Notebook 3 - Orthorectify GOES data using goes-ortho

This tests Steven's orthorectification code from goes-ortho. Copied from `goes-ortho/docs/examples
/goes-orthorectify.ipynb` on github.

The input is raw GOES ABI netcdf files and the output is corrected netcdfs. These can then be converted to zarr and processed into RGB composites as I have done with previous code (see `/processing/00_goes_rad_corrections.ipynb`)

In [1]:
import goes_ortho
import orthorectify_modded
import xarray as xr
import matplotlib.pyplot as plt
import os

import warnings
warnings.filterwarnings("ignore")

We will use the goes_ortho.ortho() function to orthorectify and clip a GOES ABI image for an area within lat/lon bounds we'll specify.

First, provide a filepath to the GOES ABI image we want to orthorectify:

In [2]:
path = '/storage/cdalden/goes/washington/goes17/2022/8/1/ABI-L1b-RadC/01/C02/'
goes_image_path = path + "OR_ABI-L1b-RadC-M6C02_G17_s20222130146177_e20222130148550_c20222130148570.nc"

We can specify a list of the variables in the dataset that we want to include in the orthorectified dataset:

In [3]:
data_vars = ["Rad"]

And then we need to provide a new filepath and filename for where to save the final orthorectified dataset:

In [4]:
new_goes_filename = path + "test_ortho.nc"

Finally, specify the lat/lon bounds for the area we want to orthorectify, like (min lon, min lat, max lon, max lat):

In [5]:
bounds = (-124, 48, -121, 49)

Run the function!

Note: goes_ortho.ortho does not exist as described in the example notebook, using goes_ortho.orthorectify.ortho instead

In [6]:
api_key = "41d14aae7e761c0de3e8f99aa4fd24d9"

orthorectify_modded.ortho(
    goes_image_path,
    data_vars,
    bounds,
    api_key,
    new_goes_filename,
    dem_filepath='temp_SRTMGL3_DEM.tif',
    demtype="SRTMGL3",
    keep_dem=True,
)

In [7]:
new_goes_filename

'/storage/cdalden/goes/washington/goes17/2022/8/1/ABI-L1b-RadC/01/C02/test_ortho.nc'

Open the new file we created, and plot it:

In [8]:
ds_ortho = xr.open_dataset(new_goes_filename)
ds_ortho

<xarray.Dataset> Size: 6MB
Dimensions:           (latitude: 251, longitude: 487)
Coordinates:
    t                 datetime64[ns] 8B ...
  * longitude         (longitude) float64 4kB -124.0 -124.0 ... -121.0 -121.0
  * latitude          (latitude) float64 2kB 49.0 49.0 48.99 ... 48.01 48.0 48.0
    dem_px_angle_x    (latitude, longitude) float64 978kB ...
    dem_px_angle_y    (latitude, longitude) float64 978kB ...
Data variables:
    elevation         (latitude, longitude) float32 489kB ...
    Rad               (latitude, longitude) float32 489kB ...
    ref               (latitude, longitude) float32 489kB ...
    abi_fixed_grid_x  (latitude, longitude) float64 978kB ...
    abi_fixed_grid_y  (latitude, longitude) float64 978kB ...
    zone_labels       (latitude, longitude) int64 978kB ...
Attributes: (12/21)
    longitude_of_projection_origin:       -137.0
    semi_major_axis:                      6378137.0
    semi_minor_axis:                      6356752.31414
    satellite_height:                     42164160.0
    grs80_eccentricity:                   0.0818191910435
    longitude_of_projection_origin_info:  longitude of geostationary satellit...
    ...                                   ...
    dem_ifov_info:                        instantaneous field of view (angula...
    dem_px_angle_x_info:                  DEM grid cell X coordinate (east/we...
    dem_px_angle_y_info:                  DEM grid cell Y coordinate (north/s...
    longitude_info:                       longitude from DEM file
    latitude_info:                        latitude from DEM file
    elevation_info:                       elevation from DEM file

In [12]:
ds = xr.open_dataset(goes_image_path)
ds

<xarray.Dataset> Size: 480MB
Dimensions:                                           (y: 6000, x: 10000,
                                                       number_of_time_bounds: 2,
                                                       number_of_image_bounds: 2,
                                                       num_star_looks: 24,
                                                       band: 1)
Coordinates:
    t                                                 datetime64[ns] 8B ...
  * y                                                 (y) float32 24kB 0.1282...
  * x                                                 (x) float32 40kB -0.069...
    y_image                                           float32 4B ...
    x_image                                           float32 4B ...
    band_id                                           (band) int8 1B ...
    band_wavelength                                   (band) float32 4B ...
    t_star_look                                       (num_star_looks) datetime64[ns] 192B ...
    band_wavelength_star_look                         (num_star_looks) float32 96B ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                num_star_looks, band
Data variables: (12/37)
    Rad                                               (y, x) float32 240MB ...
    DQF                                               (y, x) float32 240MB ...
    time_bounds                                       (number_of_time_bounds) datetime64[ns] 16B ...
    goes_imager_projection                            int32 4B ...
    y_image_bounds                                    (number_of_image_bounds) float32 8B ...
    x_image_bounds                                    (number_of_image_bounds) float32 8B ...
    ...                                                ...
    algorithm_dynamic_input_data_container            int32 4B ...
    processing_parm_version_container                 int32 4B ...
    algorithm_product_version_container               int32 4B ...
    star_id                                           (num_star_looks) float32 96B ...
    channel_integration_time                          float64 8B ...
    channel_gain_field                                float64 8B ...
Attributes: (12/30)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    standard_name_vocabulary:  CF Standard Name Table (v35, 20 July 2016)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    production_site:           RBU
    ...                        ...
    timeline_id:               ABI Mode 6
    date_created:              2022-08-01T00:28:56.9Z
    time_coverage_start:       2022-08-01T00:26:17.7Z
    time_coverage_end:         2022-08-01T00:28:55.0Z
    LUT_Filenames:             SpaceLookParams(FM2A_CDRL79RevP_PR_09_08_04)-6...
    id:                        59a151fc-695a-450b-8cc2-40e80da0bbd5

## Combine all previous work so that we can processing 1 day of data with minimal functions

1. run ortho script for all files in each day
    - this works but its really slow, need to parallize this (15s/file so potentially 3 hrs for 1 day)
2. combine all files for each day into 1 zarr per channel
3. convert to RGB values and create a zarr for all channels for each day

### Step 1 - Orthorectify raw GOES files

In [3]:
# Define the root directory to start searching
root_dir = '/storage/cdalden/goes/washington/goes17/2022/8/1/ABI-L1b-RadC/06/'

# Loop through all subdirectories and files
for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.nc'):  # Check if the file is a NetCDF file
            netcdf_path = os.path.join(subdir, file)
            print('working on... ' + netcdf_path)

            # define all args needed for the ortho function
            goes_image_path = netcdf_path
            data_vars = ["Rad"]
            new_goes_filename = netcdf_path.replace('.nc', '_ortho.nc')
            bounds = (-124, 48, -121, 49)
            api_key = "41d14aae7e761c0de3e8f99aa4fd24d9"

            orthorectify_modded.ortho(
                goes_image_path,
                data_vars,
                bounds,
                api_key,
                new_goes_filename,
                dem_filepath='temp_SRTMGL3_DEM.tif',
                demtype="SRTMGL3",
                keep_dem=True,
            )

            # Delete the original NetCDF file
            os.remove(netcdf_path)

working on... /storage/cdalden/goes/washington/goes17/2022/8/1/ABI-L1b-RadC/06/C02/OR_ABI-L1b-RadC-M6C02_G17_s20222130601177_e20222130603550_c20222130603570.nc
working on... /storage/cdalden/goes/washington/goes17/2022/8/1/ABI-L1b-RadC/06/C02/OR_ABI-L1b-RadC-M6C02_G17_s20222130606177_e20222130608550_c20222130608572.nc
working on... /storage/cdalden/goes/washington/goes17/2022/8/1/ABI-L1b-RadC/06/C02/OR_ABI-L1b-RadC-M6C02_G17_s20222130611177_e20222130613550_c20222130613569.nc
working on... /storage/cdalden/goes/washington/goes17/2022/8/1/ABI-L1b-RadC/06/C02/OR_ABI-L1b-RadC-M6C02_G17_s20222130616177_e20222130618550_c20222130618570.nc
working on... /storage/cdalden/goes/washington/goes17/2022/8/1/ABI-L1b-RadC/06/C02/OR_ABI-L1b-RadC-M6C02_G17_s20222130621177_e20222130623550_c20222130623570.nc
working on... /storage/cdalden/goes/washington/goes17/2022/8/1/ABI-L1b-RadC/06/C02/OR_ABI-L1b-RadC-M6C02_G17_s20222130626177_e20222130628550_c20222130628569.nc
working on... /storage/cdalden/goes/wash